<a href="https://colab.research.google.com/github/a2r4vind/Machine-Learning-Models/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement GAN model using TensorFlow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Generator

In [2]:
def build_generator(latent_dim):
  model = models.Sequential()
  model.add(layers.Dense(128, input_dim=latent_dim))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dense(784, activation='tanh'))
  model.add(layers.Reshape((28, 28, 1)))
  return model

# Define the Discriminator

In [3]:
def build_discriminator():
  model = models.Sequential()
  model.add(layers.Flatten(input_shape=(28, 28, 1)))
  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

# Define GAN

In [4]:
def build_gan(generator, discriminator):
  discriminator.trainable = False
  model = models.Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

# Define Loss Function

In [5]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Discriminator Loss Function

In [6]:
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

Generator Loss Function

In [7]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

Define Optimizers

In [9]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training loop

In [10]:
@tf.function
def train_step(images):
  noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)
    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training parameters

In [11]:
EPOCHS = 20
BATCH_SIZE = 64
LATENT_DIM = 100


# Create instances of generator and discriminator

In [12]:
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Create GAN model

In [13]:
gan = build_gan(generator, discriminator)

# Load and preprocess our dataset

In [ ]:
for epoch in range(EPOCHS):
  for batch in range(len(train_dataset)//BATCH_SIZE):
    images = # Load and preprocess a batch of real images from the dataset
    train_step(images)

  if epoch % 100 == 0:
    print(f'Epoch {epoch}/{EPOCHS}, Discriminator Loss: {disc_loss}, Generator Loss: {gen_loss}')